In [3]:
from shroomdk import ShroomDK
import json
import pandas as pd 

sdk = ShroomDK("5c0c5e4b-47eb-4cc2-835d-a2ddad6d1a60")


table = "optimism.velodrome.ez_lp_actions"

df_list = []


assets = ['USDC', 'WETH', 'DAI', 'sUSD', 'OP', 'FRAX', 'DOLA', 'sETH', 'MAI', 'wstETH', 'VELO']
pools = pd.read_csv('veloPools.csv')

for assetA in assets : 
    for assetB in assets : 
        if len(pools[(pools.TOKEN0_SYMBOL == assetA) & (pools.TOKEN1_SYMBOL == assetB)]) > 0 :
            pairs.append("'" + assetA + "', '" + assetB + "'")
            
pairs = []

minAmount = 10000


        

def getLPData(table, Years, Months):
    tableName = table.replace(".", "_")

    sql = f"""

        with raw as (
            select 
                ORIGIN_FROM_ADDRESS as user,
                sum(TOKEN0_AMOUNT_USD) as token0Amt,
                sum(TOKEN1_AMOUNT_USD) as token1Amt

              from optimism.velodrome.ez_lp_actions

            where TOKEN0_AMOUNT_USD > {minAmount} or TOKEN1_AMOUNT_USD > {minAmount}

            group by user 
        ) 

        SELECT * 
        FROM {table}
        WHERE  
            ORIGIN_FROM_ADDRESS IN (SELECT user FROM raw) and 
            TOKEN0_SYMBOL in ( 'USDC', 'WETH', 'DAI', 'sUSD', 'OP', 'FRAX', 'DOLA', 'sETH', 'MAI', 'wstETH', 'VELO' ) and
            TOKEN1_SYMBOL in ( 'USDC', 'WETH', 'DAI', 'sUSD', 'OP', 'FRAX', 'DOLA', 'sETH', 'MAI', 'wstETH', 'VELO' ) 


    """

    results = sdk.query(sql)
    data = json.loads(results.json())
    df = pd.DataFrame(data['records'])
    print("Total Records : " + str(len(df)))

        
    return df 
    #finalDf.to_csv(tableName + ".csv")

In [4]:
df = getLPData(table, Years, Months)
df.to_csv("VeloLPData.csv")

Total Records : 23278


In [6]:
def getTradeData():
    print("Grabbing Trade Data For Velo" )
    sql = f"""
        select 
            min(BLOCK_TIMESTAMP) as first_trade,
            max(BLOCK_TIMESTAMP) as last_trade,
            sum(AMOUNT_IN_USD) as totalVolume, 
            avg(AMOUNT_IN_USD) as avgTradeSize,
            count(*) as total_trades,  
            ORIGIN_FROM_ADDRESS as user, 
            LP_FEE_SYMBOL as pair

        from optimism.velodrome.ez_swaps
        
        where AMOUNT_IN_USD > 5000 
        
        group by 
          user,
          pair 

    """

    results = sdk.query(sql)
    data = json.loads(results.json())
    df = pd.DataFrame(data['records'])
    print("Total Records : " + str(len(df)))

        
    #finalDf = pd.concat(df_list)    
    return df 
    #finalDf.to_csv(tableName + ".csv")

In [7]:
tradeDf = getTradeData()
tradeDf.to_csv("tradeData.csv")

Grabbing Trade Data For Velo
Total Records : 7656
